In [20]:
import os
import json
import boto3

import pandas as pd

from dotenv import load_dotenv
from sqlalchemy import create_engine, text

load_dotenv()

ACCESS_KEY = os.getenv("ID_KEY_ACCESS")
ACCESS_KEY_SECRETE = os.getenv("KEY_SECRET")
BUCKET_NAME = os.getenv("BUCKETNAME")

In [11]:
df_1 = pd.read_json("../hotel_1.json")
df_2 = pd.read_json("../hotel_2.json")
df_3 = pd.read_json("../temps.json")

In [12]:
def merge_outer(data_1:pd.DataFrame, data_2:pd.DataFrame, on:str) -> pd.DataFrame:
    df = pd.merge(data_1, data_2, how='outer', on=on)
    return df

df = merge_outer(df_1, df_2, "hotel")
df = merge_outer(df, df_3, "ville_rechercher")

In [13]:
csv = df.to_csv()
df.to_csv("hotel.csv")

In [ ]:
session = boto3.Session(aws_access_key_id=ACCESS_KEY, 
                        aws_secret_access_key=ACCESS_KEY_SECRETE)
s3 = session.resource("s3")

bucket = s3.create_bucket(Bucket=BUCKET_NAME, CreateBucketConfiguration={'LocationConstraint': 'eu-west-3'})

put_object = bucket.put_object(Key="hotel.csv", Body=csv)

In [ ]:
s3.Bucket(BUCKET_NAME).upload_file('hotel.csv', 'hotel.csv')

In [16]:
df = pd.read_csv("hotel.csv")

In [17]:
# engine = create_engine(f"postgresql+psycopg2://{YOUR_USERNAME}:{YOUR_PASSWORD}@{YOUR_HOSTNAME}/postgres", echo=True)
engine = create_engine('sqlite:///:memory:', echo=True)


In [18]:
df.to_sql("hotel", engine)

2022-12-04 12:10:43,088 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("hotel")
2022-12-04 12:10:43,088 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-04 12:10:43,089 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("hotel")
2022-12-04 12:10:43,089 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-04 12:10:43,091 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-04 12:10:43,092 INFO sqlalchemy.engine.Engine 
CREATE TABLE hotel (
	"index" BIGINT, 
	"Unnamed: 0" BIGINT, 
	ville_rechercher TEXT, 
	"ville_précise" TEXT, 
	hotel TEXT, 
	url TEXT, 
	note TEXT, 
	description TEXT, 
	latitutde FLOAT, 
	longitude FLOAT, 
	"2022-12-04 15:00:00" TEXT, 
	"2022-12-05 15:00:00" TEXT, 
	"2022-12-06 15:00:00" TEXT, 
	"2022-12-07 15:00:00" TEXT, 
	"2022-12-08 15:00:00" TEXT
)


2022-12-04 12:10:43,092 INFO sqlalchemy.engine.Engine [no key 0.00038s] ()
2022-12-04 12:10:43,093 INFO sqlalchemy.engine.Engine CREATE INDEX ix_hotel_index ON hotel ("index")
2022-12-04 12:10:43,093 INFO sql

878

In [19]:
conn = engine.connect()

In [21]:
statement = text("SELECT DISTINCT ville_précise FROM hotel")

In [22]:
result = conn.execute(statement)

2022-12-04 12:12:18,073 INFO sqlalchemy.engine.Engine SELECT DISTINCT ville_précise FROM hotel
2022-12-04 12:12:18,073 INFO sqlalchemy.engine.Engine [generated in 0.00067s] ()


In [23]:
result.fetchall()

[('Le Mont-Saint-Michel',),
 ('Ardevon',),
 ('Pontorson',),
 ('Beauvoir',),
 ('Saint-Malo',),
 ('Paramé, Saint-Malo',),
 ('Intra muros, Saint-Malo',),
 ('Saint-Servan, Saint-Malo',),
 ('Sillon, Saint-Malo',),
 ('Bayeux',),
 ('Centre-ville de Rouen, Rouen',),
 ('Rouen',),
 ('Perret, Le Havre',),
 ('Le Havre',),
 ('6e arr., Paris',),
 ('13e arr., Paris',),
 ('9e arr., Paris',),
 ('5e arr., Paris',),
 ('10e arr., Paris',),
 ('17e arr., Paris',),
 ('14e arr., Paris',),
 ('16e arr., Paris',),
 ('3e arr., Paris',),
 ('15e arr., Paris',),
 ('11e arr., Paris',),
 ('8e arr., Paris',),
 ('19e arr., Paris',),
 ('2e arr., Paris',),
 ('12e arr., Paris',),
 ('Amiens',),
 ('Centre de Lille, Lille',),
 ('Lille Sud, Lille',),
 ('Vieux-Lille, Lille',),
 ('Vauban, Lille',),
 ('Lille',),
 ('Faubourg de Béthune, Lille',),
 ('Wazemmes, Lille',),
 ('Fives, Lille',),
 ('Centre de Strasbourg - Petite France - Cathédrale, Strasbourg',),
 ('Strasbourg',),
 ('Quartier Européen - Orangerie, Strasbourg',),
 ('Neudo